In [1]:
import time
import pyautogui
from object.database_connection import DatabaseConnection
from object.dofus_agent import Agent
from object.map_state import MapState
from utils_dofus import ACTIONS, get_player_pos, dict_swap_pos_to_array_index, dict_swap_pos_to_dofus, MONSTER, INVO, BOSS, END_TURN, base_x, base_y, step_y, step_x, box_size, USABLE_CELL, PLAYER, count_pm_to_pos, dict_swap_array_index_to_dofus
import arcade
import os

In [2]:
database = DatabaseConnection()
agent = Agent(database, False)

In [3]:
def can_play(screen):
    try:
        if pyautogui.locate("resources/end_turn.png", screen) is not None:
            return True
    except pyautogui.ImageNotFoundException:
        return False
    return False

In [4]:
def get_pa(screen):
    try:
        if pyautogui.locate("resources/pa_6.png", screen) is not None:
            return 6
        if pyautogui.locate("resources/pa_3.png", screen) is not None:
            return 3
        if pyautogui.locate("resources/pa_0.png", screen) is not None:
            return 0
    except pyautogui.ImageNotFoundException:
        return 0
    return 0

In [5]:
def get_pm(screen):
    try:
        if pyautogui.locate("resources/pm_3.png", screen) is not None:
            return 3
        if pyautogui.locate("resources/pm_2.png", screen) is not None:
            return 2
        if pyautogui.locate("resources/pm_1.png", screen) is not None:
            return 1
        if pyautogui.locate("resources/pm_0.png", screen) is not None:
            return 0
    except pyautogui.ImageNotFoundException:
        return 0
    return 0

In [6]:
def center_cell(dofus_pos):
    x = base_x + step_x * dofus_pos[0]
    y = base_y + step_y * dofus_pos[1]
    
    if dofus_pos[2]:
        x += step_x // 2
        y += step_y // 2
        
    x += box_size // 2
    y += box_size
    
    return x, y

In [7]:
def do_for_real(state, action):

    clicked = ACTIONS[action]
    player_pos = get_player_pos(state.map)

    if player_pos == -1:
        return None
    
    if clicked == END_TURN:
        pyautogui.keyDown("f1")
        time.sleep(0.01)
        pyautogui.keyUp("f1")
        return None

    clicked_pos = player_pos[0] + clicked[0], player_pos[1] + clicked[1]

    if clicked_pos not in dict_swap_pos_to_array_index.keys():
        return None
    
    clicked_index = dict_swap_pos_to_array_index[clicked_pos]
    
    if state.map[clicked_index] == MONSTER or state.map[clicked_index] == INVO or state.map[clicked_index] == BOSS:
        state.map[clicked_index] = USABLE_CELL
        pyautogui.keyDown("2")
        time.sleep(0.01)
        pyautogui.keyUp("2")
        time.sleep(0.1)
    elif state.map[clicked_index] == USABLE_CELL:
        pm_used = count_pm_to_pos(state.map, player_pos, clicked_pos, state.pm)
        if pm_used == -1 or pm_used > state.pm:
            return None
        state.pm -= pm_used        
        state.map[clicked_index] = PLAYER
        state.map[dict_swap_pos_to_array_index[player_pos]] = USABLE_CELL

    pos = center_cell(dict_swap_pos_to_dofus[clicked_pos])
    pyautogui.click(pos)
    pyautogui.moveTo(100, 100, duration=0.1)
    #move_mouse(pos[0], pos[1])
    #click()
    #move_mouse(100, 100)
    
    return state

In [8]:
class DofusStatusWindow(arcade.Window):
    
    def __init__(self):
        super().__init__(2256 // 2, 1504 // 2, "DofusStatus")
        arcade.set_background_color(arcade.color.BLACK)
        self.map_state = None
        self.action = None
        
    def draw_box(self, x, y, color):
        arcade.draw_rectangle_filled(x, 1504 // 2 - y, box_size // 2, box_size // 2, color)
        
    def draw_box_outline(self, x, y, color):
        arcade.draw_rectangle_outline(x, 1504 // 2 - y, box_size // 2, box_size // 2, color, border_width=5)    
        
    def on_draw(self):
        arcade.start_render()
        if self.map_state is not None:
            for pos in range(len(self.map_state.map)):
                if self.map_state.map[pos] != 0:
    
                    dofus_pos = dict_swap_array_index_to_dofus[pos]
    
                    x = base_x + step_x * dofus_pos[0]
                    y = base_y + step_y * dofus_pos[1]
                    
                    if dofus_pos[2]:
                        x += step_x // 2
                        y += step_y // 2
                    
                    x = x // 2
                    y = y // 2
                    
                    if self.map_state.map[pos] == 1:
                        self.draw_box(x, y, arcade.color.GRAY)
                    elif self.map_state.map[pos] == 2:
                        self.draw_box(x, y, arcade.color.GREEN)
                    elif self.map_state.map[pos] == 3:
                        self.draw_box(x, y, arcade.color.BLUE)
                    elif self.map_state.map[pos] == 4:
                        self.draw_box(x, y, arcade.color.RED)
                    elif self.map_state.map[pos] == 5:
                        self.draw_box(x, y, arcade.color.YELLOW)
                    elif self.map_state.map[pos] == 6:
                        self.draw_box(x, y, arcade.color.ORANGE)
                    else:
                        self.draw_box(x, y, arcade.color.AQUA)

            if self.action is not None and ACTIONS[self.action] != END_TURN:
                
                clicked = ACTIONS[self.action]
                player_pos = get_player_pos(self.map_state.map)
            
                if player_pos == -1:
                    return
                
                clicked_pos = player_pos[0] + clicked[0], player_pos[1] + clicked[1]
                
                dofus_pos = dict_swap_pos_to_dofus[clicked_pos]
        
                x = base_x + step_x * dofus_pos[0]
                y = base_y + step_y * dofus_pos[1]
                
                if dofus_pos[2]:
                    x += step_x // 2
                    y += step_y // 2
                
                x = x // 2
                y = y // 2
                
                self.draw_box_outline(x, y, arcade.color.WHITE)

    def on_update(self, delta_time):
        time.sleep(0.5)
        if self.map_state is None:
            screenshot = pyautogui.screenshot()
            if can_play(screenshot):
                self.map_state = MapState.screen_info(get_pa(screenshot), get_pm(screenshot), screenshot=screenshot)
            else:
                print("Searching for action to do")
        else:
            self.action, _ = agent.best_action(self.map_state)
            self.map_state = do_for_real(self.map_state, self.action)


In [9]:
#window = DofusStatusWindow()
#arcade.run()

In [10]:
while 1:
    time.sleep(3)
    screenshot = pyautogui.screenshot()
    if can_play(screenshot):
        time.sleep(1)
        screenshot = pyautogui.screenshot()
        map_state = MapState.screen_info(get_pa(screenshot), get_pm(screenshot), screenshot=screenshot)
        while map_state is not None:
            action, _ = agent.best_action(map_state)
            map_state = do_for_real(map_state, action)
            time.sleep(1)
    else:
        print("Searching for action to do")

KeyboardInterrupt: 